In [1]:
#!pip install simple_fpa

In [2]:
%%capture out
! cd ..; pip install .

In [3]:
print((out.stdout.split('\n')[-2]))

In [4]:
from simple_fpa import Model, Simulator
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
from scipy.stats import beta

rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Arial"],
    "lines.linewidth": 1,
    "figure.dpi":200
})

import warnings
warnings.filterwarnings("ignore")

In [5]:
sample_size = 1000000
smoothing_rate = 0.3
trim_percent = 2
rv = beta(2,2)
eps = 0.01
frec = {3:1}

In [6]:
sim = Simulator(sample_size, smoothing_rate, trim_percent, frec, rv.pdf, rv.ppf, eps, reflect = True)
sim.calibrate()
sim.make_true()

In [7]:
%time sim.simulate_uni(draws = 1000, nominal_coverage = 95)

CPU times: user 6.2 s, sys: 10.7 s, total: 16.9 s
Wall time: 1min 32s


In [8]:
#sim.crit_qs_uni

In [9]:
%time sim.simulate_dgp(draws = 1000, nominal_coverage = 95)

CPU times: user 4.66 s, sys: 7.55 s, total: 12.2 s
Wall time: 5min 40s


In [10]:
for x,y in zip(['v','bs','rev','rev2','ts','ts2'],sim.cov):
    print('{:3} : {:}'.format(x,y))

v   : 0.948
bs  : 0.969
rev : 0.951
rev2 : 0.957
ts  : 0.996
ts2 : 0.95
